# Unit 2.2 - Secure System Integration

## Lab Step 1: Load the optimized model securely 

In this first lab step, you will:

1. Load the optimized HAR model (pruned + quantized) using ONNX Runtime.  
2. Inspect the model’s input and output tensors.  
3. Run a minimal “smoke test” inference to confirm that the model is functional.

This step establishes trust in the model artifact before we build the secure inference pipeline around it.

### 1.1 - Task 1: Import dependencies and load the model

In this task you will:

- Configure the Python path so the notebook can import modules from the `src` folder.
- Use the `load_model` helper function to load the optimized ONNX model.
- Confirm that the ONNX Runtime session is created successfully.

In [1]:
import os
import sys

# Ensure that the `src` folder is on the Python path
# (this assumes the notebook is inside the `notebooks` folder)
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")

if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print("Project root:", PROJECT_ROOT)
print("Source dir  :", SRC_DIR)
print("Models dir  :", MODELS_DIR)

# Import the helper for loading ONNX models
from load_model import load_model

# Path to the optimized model (produced in Unit 2.1)
MODEL_PATH = os.path.join(MODELS_DIR, "har_pruned_quantized.onnx")
print("Model path  :", MODEL_PATH)

# Load the model using ONNX Runtime
session, input_tensor, output_tensor = load_model(MODEPATH := MODEL_PATH)

print("Model loaded successfully.")


Project root: C:\Users\ikybe\5g-digits\unit22
Source dir  : C:\Users\ikybe\5g-digits\unit22\src
Models dir  : C:\Users\ikybe\5g-digits\unit22\models
Model path  : C:\Users\ikybe\5g-digits\unit22\models\har_pruned_quantized.onnx
Model loaded successfully.


### 1.2 - Task 2: Inspect input and output tensors

Now that the model is loaded, we need to understand **how** it expects data:

- What is the **input tensor name**?
- What is the **input tensor shape** (batch size, window length, number of channels)?
- What is the **data type** of the inputs and outputs?

This information will be used later to implement the input validation functions.

In [2]:
# Inspect input tensor
input_name = input_tensor.name
input_shape = input_tensor.shape
input_type = input_tensor.type

# Inspect output tensor
output_name = output_tensor.name
output_shape = output_tensor.shape
output_type = output_tensor.type

print("=== Input tensor ===")
print("Name :", input_name)
print("Shape:", input_shape)
print("Type :", input_type)
print()
print("=== Output tensor ===")
print("Name :", output_name)
print("Shape:", output_shape)
print("Type :", output_type)

=== Input tensor ===
Name : input
Shape: ['unk__121', 100, 7]
Type : tensor(float)

=== Output tensor ===
Name : dense_1
Shape: ['unk__122', 8]
Type : tensor(float)


### 1.3 - Task 3: Run a minimal smoke test inference

Before integrating the model into a secure pipeline, we perform a **smoke test**:

- Create a dummy input window with the correct shape and data type.
- Run a single inference through the model.
- Check that the model returns a valid output tensor (no errors or exceptions).

This confirms that the model file is not corrupted and that ONNX Runtime can execute it.

In [3]:
import numpy as np
import onnxruntime as ort

# Create a dummy input window based on the input shape
# input_shape typically looks like: [None, window_length, num_channels]
# We create a single batch (1, W, C) with reasonable IMU value ranges.

batch_size = 1
window_length = input_shape[1]
num_channels = input_shape[2]

dummy_window = np.zeros((batch_size, window_length, num_channels), dtype=np.float32)

# Fill with small random values in a plausible IMU range, e.g. [-1.0, 1.0]
dummy_window = np.random.uniform(-1.0, 1.0, size=dummy_window.shape).astype(np.float32)

# Run a smoke test inference
outputs = session.run([output_name], {input_name: dummy_window})
predictions = outputs[0]

print("Smoke test completed successfully.")
print("Output shape:", predictions.shape)
print("Sample output (first row):", predictions[0])


Smoke test completed successfully.
Output shape: (1, 8)
Sample output (first row): [3.5683908e-02 9.3733311e-02 4.6272118e-02 5.2003423e-04 6.4002134e-02
 5.4000902e-01 2.1881647e-01 9.6306583e-04]


#### 1.3.1. Reflection

If this smoke test fails (for example, due to a shape mismatch or runtime error),  
what would be the most appropriate next step?

- Inspect the model’s input shape and ensure the dummy window matches it.  
- Check that the model file path is correct.  
- Verify that ONNX Runtime is installed and working.

These checks must be completed before proceeding to input validation and secure integration.

## Lab Step 2 – Build the Secure Input Pipeline

In this step, you will implement the fundamental validation layers required for secure integration of a model on an edge device. The goal is to ensure that only valid, meaningful, and physically plausible data reaches the inference engine.

This step is divided into five parts:

2.1 Shape Validation  
2.2 Type Validation  
2.3 Range Validation  
2.4 Combined Validator  
2.5 Testing Invalid Cases

### 2.1 - Shape Validation

The model expects input windows with a specific shape:
(batch_size = 1, window_length, num_channels)

This shape must match exactly what the model’s input tensor specifies.  
If the shape is incorrect, ONNX Runtime will fail immediately.

In this section, you will implement a function that verifies the shape of an input window before it reaches the model.


In [4]:
from validation import validate_shape

expected_shape = tuple(input_shape)  # From Lab Step 1

def test_shape(window):
    return validate_shape(window, expected_shape)

print("Expected shape:", expected_shape)

Expected shape: ('unk__121', 100, 7)


### 2.2 – Type Validation

Even if the shape is correct, the input window must also have the correct data type.  
This model expects float32 (`np.float32`) inputs.

Here you will implement type validation to ensure dtype consistency.

In [5]:
from validation import validate_dtype
import numpy as np

expected_dtype = np.float32

def test_dtype(window):
    return validate_dtype(window, expected_dtype)

print("Expected dtype:", expected_dtype)


Expected dtype: <class 'numpy.float32'>


### 2.3 - Range Validation

Physical sensors such as IMUs operate within well-defined numerical limits.  
Values outside the allowed range indicate noise, corruption, or sensor malfunction.

For this model, we assume accelerometer and gyroscope values must remain within:
[-4.0, +4.0]

Here you will implement range validation to detect out-of-bound values.

In [6]:
from validation import validate_range

min_val, max_val = -4.0, 4.0

def test_range(window):
    return validate_range(window, min_value=min_val, max_value=max_val)

print(f"Valid range: {min_val} to {max_val}")

Valid range: -4.0 to 4.0


## 2.4 - Combined Validator (Master Function)

Now that we have implemented shape, type, and range checks individually,  
we will combine them into a single master validator.

If any component fails, the whole validation fails.

This validator will be used later in the secure inference wrapper.


In [7]:
from validation import validate_input

def master_validate(window):
    return validate_input(
        window,
        expected_shape=expected_shape,
        expected_dtype=expected_dtype
    )

print("Master validator ready.")


Master validator ready.


### 2.5 - Testing Invalid Cases

A validator is only reliable if it rejects invalid inputs *consistently*.  
In this section, you will test the validator with deliberately malformed inputs:

- Wrong shapes  
- Wrong dtype  
- Out-of-range values  
- NaN or infinite values  
- Corrupted windows (missing elements)

Note:
ONNX models sometimes contain symbolic (non-numeric) dimension names such as "unk__120" instead of actual integers.
This is normal: symbolic dimensions indicate that the model accepts variable-length input sizes.
However, NumPy cannot create arrays using symbolic dimensions.
Therefore, when generating synthetic test inputs, replace any symbolic dimension with a numeric placeholder

In [8]:
import numpy as np

# Convert expected_shape to usable numeric shape
numeric_shape = tuple(
    dim if isinstance(dim, int) else 1   # replace strings like "unk__120" by 1
    for dim in expected_shape
)

print("Numeric shape used for validation tests:", numeric_shape)

invalid_shape = np.zeros((1, 50, 3), dtype=np.float32)  # wrong window length
invalid_dtype = np.zeros(numeric_shape, dtype=np.float64)  # wrong dtype
invalid_range = np.random.uniform(-10, 10, size=numeric_shape).astype(np.float32)
invalid_nan = np.zeros(numeric_shape, dtype=np.float32); invalid_nan[0, 0, 0] = np.nan

tests = {
    "Wrong shape": invalid_shape,
    "Wrong dtype": invalid_dtype,
    "Out of range": invalid_range,
    "Contains NaN": invalid_nan,
}

for name, w in tests.items():
    print(f"{name}: {master_validate(w)}")


Numeric shape used for validation tests: (1, 100, 7)
Wrong shape: False
Wrong dtype: False
Out of range: False
Contains NaN: False


## Lab Step 3:  Safe Inference Wrapper

### 3.1 - What Is a Safe Inference Wrapper?

In real embedded deployments, raw data cannot be trusted.  
Sensors may fail, values may drift, and communication channels may introduce corruption.

A **Safe Inference Wrapper** is a protective function that:

- Validates the input tensor
- Rejects unsafe inputs early
- Logs structured errors
- Only runs inference when validation succeeds

This wrapper will become the main interface for secure model execution inside an embedded AI system.


### 3.2 – Implementing the Safe Inference Wrapper

The safe inference wrapper will:

1. Validate the input window using the combined validator.
2. Only run inference if validation passes.
3. Return a **structured dictionary** with:
   - `ok` (True/False)
   - `error` (string or None)
   - `prediction` (class index or None)

This function will be the main secure entry point for running the model.

In [9]:
def secure_predict(session, input_name, output_name, window, verbose: bool = False):
    """
    Safe inference wrapper for the optimized HAR model.

    Parameters
    ----------
    session : onnxruntime.InferenceSession
        ONNX Runtime session for the model.
    input_name : str
        Name of the model's input tensor.
    output_name : str
        Name of the model's output tensor.
    window : np.ndarray
        Input window to validate and pass to the model.
    verbose : bool
        If True, print diagnostic messages.

    Returns
    -------
    dict with keys:
        - "ok": bool
        - "error": str or None
        - "prediction": int or None
    """

    # 1) Validation
    is_valid = master_validate(window)

    if not is_valid:
        if verbose:
            print("[secure_predict] Validation failed.")
        return {
            "ok": False,
            "error": "validation_failed",
            "prediction": None,
        }

    # 2) Prepare input shape (ensure batch dimension)
    try:
        if window.ndim == 2:
            # (W, C) -> (1, W, C)
            window_batch = window[None, ...]
        elif window.ndim == 3:
            window_batch = window
        else:
            if verbose:
                print("[secure_predict] Unexpected input ndim:", window.ndim)
            return {
                "ok": False,
                "error": "unexpected_ndim",
                "prediction": None,
            }

        # 3) Inference
        outputs = session.run([output_name], {input_name: window_batch})
        preds = outputs[0]

        # Handle typical (1, num_classes) output
        if preds.ndim == 2:
            pred_idx = int(preds[0].argmax())
        else:
            pred_idx = int(preds.argmax())

        return {
            "ok": True,
            "error": None,
            "prediction": pred_idx,
        }

    except Exception as e:
        if verbose:
            print("[secure_predict] Runtime error:", repr(e))
        return {
            "ok": False,
            "error": "runtime_error",
            "prediction": None,
        }


### 3.3 - Testing the Safe Inference Wrapper

To test `secure_predict`, we generate:

1. A **synthetic valid window** with the correct shape and value range.
2. The **invalid windows** created in Lab Step 2.5.

The goal is to confirm:

- `ok = True` and a valid `prediction` for the correct input.
- `ok = False`, a meaningful `error`, and `prediction = None` for invalid inputs.


In [11]:
valid_window = np.random.uniform(
    low=-1.0, high=1.0,
    size=(window_length, num_channels)
).astype(np.float32)

print("Valid window shape:", valid_window.shape)
print("Valid window dtype:", valid_window.dtype)

result_valid = secure_predict(session, input_name, output_name, valid_window, verbose=True)
print("Valid input result:", result_valid)



Valid window shape: (100, 7)
Valid window dtype: float32
Valid input result: {'ok': True, 'error': None, 'prediction': 1}


### 3.4 - Testing Invalid Inputs (Robustness Verification)

In this step, we verify that the safe inference wrapper correctly handles invalid inputs.  
We test several failure modes: wrong shape, wrong dtype, out-of-range values, and NaNs.

A valid system should reject each invalid input, return a clear error message, and avoid running inference.


In [12]:
# invalid_shape
invalid_shape = np.zeros((1, window_length - 50, num_channels), dtype=np.float32)

# invalid_dtype
invalid_dtype = np.zeros((window_length, num_channels), dtype=np.float64)

# invalid_range
invalid_range = np.random.uniform(-10, 10, size=(window_length, num_channels)).astype(np.float32)

# invalid_nan
invalid_nan = np.random.uniform(-1, 1, size=(window_length, num_channels)).astype(np.float32)
invalid_nan[0, 0] = np.nan


tests = {
    "Wrong shape": invalid_shape,
    "Wrong dtype": invalid_dtype,
    "Out of range": invalid_range,
    "Contains NaN": invalid_nan,
}

print("Running invalid input tests...\n")

for name, w in tests.items():
    print(f"--- {name} ---")
    result = secure_predict(session, input_name, output_name, w, verbose=True)
    print(result)
    print()


Running invalid input tests...

--- Wrong shape ---
[secure_predict] Validation failed.
{'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Wrong dtype ---
[secure_predict] Validation failed.
{'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Out of range ---
[secure_predict] Validation failed.
{'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Contains NaN ---
[secure_predict] Validation failed.
{'ok': False, 'error': 'validation_failed', 'prediction': None}



#### Observations

- All invalid inputs should be rejected (`ok=False`).
- The system should return a clear and meaningful error code.
- No prediction should be produced for invalid inputs.
- This validates that the wrapper enforces "validation before inference," an essential principle for safety.


## Lab Step 4 - Designing a Safe On-Device Inference API

In the previous lab steps you implemented input validation and a secure inference wrapper.  
We now look at how these components come together to form a **Safe On-Device Inference API**.

This API acts as a protective boundary between the application and the model, ensuring that only valid inputs reach the inference engine and that failures are handled in a structured way.

### 4.1 - Why a Safe On-Device Inference API?

Edge devices operate in environments where sensor data may be noisy, incomplete, or corrupted.  
A safe API ensures that:

- invalid inputs are rejected
- inference is only executed when safe
- applications receive predictable, structured responses

This layer becomes the main entry point for calling the model inside an embedded or mobile system.


### 4.2 - API Structure and Contract

A Safe Inference API defines a clear contract about:

**1. What inputs are acceptable**
- correct shape
- correct data type
- physically plausible values
- no NaNs or infinities

**2. What outputs look like**
- structured dictionary (`ok`, `error`, `prediction`)
- no silent failures

**3. How errors are reported**
- explicit, predictable error messages
- no exceptions surfacing to the application

This contract makes the system reliable and easy to integrate.



### 4.3 - Handling Errors Consistently

A safe API must behave predictably when inputs are invalid.

Instead of:
- raising exceptions, or
- producing undefined results,

the API returns a *clear error signal* that the application can act on.

Consistent error handling is essential for robustness in real environments, where sensor disturbances or transmission issues may occur frequently.




### 4.4 - What a Safe Prediction Flow Looks Like

A safe prediction flow follows these steps:

1. **Collect input** (sensor window)
2. **Validate the window** before inference
3. **Decision:**
   - If valid → run inference
   - If invalid → return an error, no prediction
4. **Produce structured output**

This ensures the model is only called under safe, known conditions.


### 4.5 – How the Implementation Will Work (Conceptual)

In the next sections of the notebook you will assemble:

- the validation functions
- the secure prediction wrapper
- a clean API interface

The API will expose a single call that applications can use safely, without knowing the internal details of validation or inference.

This demonstrates how AI components can be packaged for reliable on-device use.


## 5. Lab Step 5 - Implementing the Safe Inference API

In this step, we turn the concepts from Section 7 into a working Python API.

The API will:
- validate inputs internally
- handle errors consistently
- run inference only when safe
- return structured results with clear success/failure signals

We implement the API as a Python class, supported by the validation functions and the secure prediction wrapper developed earlier.


### 5.1 – API Structure

We implement the Safe Inference API as a Python class:

- The class stores:
  - the ONNX inference session
  - the model's input/output tensor names
  - expected shapes and data types

- It exposes a single method:
  `predict(window)`

- Internally, it integrates:
  - our master validator
  - the secure prediction wrapper
  - structured error communication

This gives the application a clean and predictable interface.


In [14]:
class SafeInferenceAPI:
    """
    High-level API for safe, validated on-device inference.
    Wraps validation + secure_predict into a simple interface.
    """

    def __init__(self, session, input_name, output_name, expected_shape, expected_dtype):
        self.session = session
        self.input_name = input_name
        self.output_name = output_name
        self.expected_shape = expected_shape
        self.expected_dtype = expected_dtype

    def predict(self, window, verbose: bool = False):
        """
        Main public method: validate input and run secure prediction.
        Returns a structured dictionary.
        """

        # 1) Input validation
        is_valid = validate_input(
            window,
            expected_shape=self.expected_shape,
            expected_dtype=self.expected_dtype
        )

        if not is_valid:
            if verbose:
                print("[SafeInferenceAPI] Validation failed.")
            return {
                "ok": False,
                "error": "validation_failed",
                "prediction": None
            }

        # 2) Secure inference
        result = secure_predict(
            self.session, 
            self.input_name, 
            self.output_name, 
            window, 
            verbose=verbose
        )

        return result


### 5.2 - Creating an Instance of the API

We now create an API object using:

- our ONNX Runtime session
- the model input/output tensor names
- the expected input shape and dtype detected earlier

This turns the model into a reusable, safe component.


In [15]:
api = SafeInferenceAPI(
    session=session,
    input_name=input_name,
    output_name=output_name,
    expected_shape=expected_shape,
    expected_dtype=expected_dtype
)

print("SafeInferenceAPI instance created.")


SafeInferenceAPI instance created.


### 5.3 – Test the API with a Valid Window

We reuse a synthetic valid window to check the API’s behaviour.
The API should return:

- ok = True  
- error = None  
- prediction = <class index>  


In [16]:
result_valid = api.predict(valid_window, verbose=True)
print("API result (valid input):", result_valid)


API result (valid input): {'ok': True, 'error': None, 'prediction': 1}


### 5.4 – Test the API with Invalid Inputs

We now test the API with the intentionally malformed windows created earlier:

- wrong shape  
- wrong dtype  
- out-of-range values  
- values containing NaNs  

All cases should:

- fail validation  
- return `ok = False`  
- produce an appropriate error  
- never run inference  


In [17]:
tests = {
    "Wrong shape": invalid_shape,
    "Wrong dtype": invalid_dtype,
    "Out of range": invalid_range,
    "Contains NaN": invalid_nan,
}

for name, w in tests.items():
    print(f"\n--- {name} ---")
    result = api.predict(w, verbose=True)
    print("API result:", result)



--- Wrong shape ---
[SafeInferenceAPI] Validation failed.
API result: {'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Wrong dtype ---
[SafeInferenceAPI] Validation failed.
API result: {'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Out of range ---
[SafeInferenceAPI] Validation failed.
API result: {'ok': False, 'error': 'validation_failed', 'prediction': None}

--- Contains NaN ---
[SafeInferenceAPI] Validation failed.
API result: {'ok': False, 'error': 'validation_failed', 'prediction': None}


### Reflection

In your own words:

- What protections does this API add compared to calling the model directly?
- How does structured error reporting make application code more reliable?
- Why is it important that the API exposes only *one* method (`predict`) for the application?

Write a few sentences summarizing your insights.


## 6. Lab Step 6 - Running the Complete Secure Pipeline

In this final step, we run the entire secure prediction pipeline as a single system.
This includes:

- input window creation
- validation
- safe inference
- structured output reporting

This demonstrates how the Safe Inference API behaves under normal and abnormal operating conditions.


### 6.1 - Running the Safe API on a Valid Input

We start by running the API on a valid sensor window.
The expected result is:

- `ok = True`
- `error = None`
- a numerical prediction (class index)


In [18]:
print("Running pipeline test with a valid window...\n")

result_valid_pipeline = api.predict(valid_window, verbose=True)
print("Pipeline result (valid input):")
print(result_valid_pipeline)


Running pipeline test with a valid window...

Pipeline result (valid input):
{'ok': True, 'error': None, 'prediction': 1}


### 6.2 - Running the Safe API on Invalid Inputs

We now test the pipeline with several invalid windows:

- wrong shape  
- wrong dtype  
- out-of-range values  
- values containing NaNs  

The pipeline should:

- reject the input (`ok = False`)
- produce a clear error message
- avoid running inference


In [19]:
print("\nRunning pipeline tests with invalid inputs...\n")

tests = {
    "Wrong shape": invalid_shape,
    "Wrong dtype": invalid_dtype,
    "Out of range": invalid_range,
    "Contains NaN": invalid_nan,
}

pipeline_results = {}

for name, w in tests.items():
    print(f"--- {name} ---")
    result = api.predict(w, verbose=True)
    pipeline_results[name] = result
    print("Pipeline result:", result, "\n")



Running pipeline tests with invalid inputs...

--- Wrong shape ---
[SafeInferenceAPI] Validation failed.
Pipeline result: {'ok': False, 'error': 'validation_failed', 'prediction': None} 

--- Wrong dtype ---
[SafeInferenceAPI] Validation failed.
Pipeline result: {'ok': False, 'error': 'validation_failed', 'prediction': None} 

--- Out of range ---
[SafeInferenceAPI] Validation failed.
Pipeline result: {'ok': False, 'error': 'validation_failed', 'prediction': None} 

--- Contains NaN ---
[SafeInferenceAPI] Validation failed.
Pipeline result: {'ok': False, 'error': 'validation_failed', 'prediction': None} 



### 6.3 - Summary of Pipeline Behaviour

Below we produce a simple summary table comparing:

- valid input behaviour  
- invalid input behaviour  

This helps visualize how the Safe Inference API responds under different conditions.


In [20]:
summary = {
    "Valid Input": result_valid_pipeline
}

summary.update(pipeline_results)

print("\n--- Pipeline Summary ---\n")
for key, value in summary.items():
    print(f"{key}: {value}")



--- Pipeline Summary ---

Valid Input: {'ok': True, 'error': None, 'prediction': 1}
Wrong shape: {'ok': False, 'error': 'validation_failed', 'prediction': None}
Wrong dtype: {'ok': False, 'error': 'validation_failed', 'prediction': None}
Out of range: {'ok': False, 'error': 'validation_failed', 'prediction': None}
Contains NaN: {'ok': False, 'error': 'validation_failed', 'prediction': None}


### Final Reflection

After completing this step:

- How does the Safe Inference API protect the model?
- What differences do you observe between valid and invalid inference paths?
- How could this design be extended for real deployment (e.g., logging, rate limiting, metadata)?

Write a short reflection summarizing your understanding.
